In [12]:
from pymilvus import connections, Collection

connections.connect(alias="default", host="localhost", port="19530")

In [13]:
collection = Collection("embedded_music_data")

In [ ]:
collection.load()

In [ ]:
# Get the embedding for a song by name from the collection
song_name = "Shape of You"  # Replace with any song name in your DB

# Query for the song's embedding
results = collection.query(expr=f'name == "{song_name}"', output_fields=["embedding"])
if not results:
    raise ValueError(f"Song '{song_name}' not found in the collection.")

query_embedding = results[0]["embedding"]

# Search for 5 most similar songs using the embedding
search_results = collection.search(
    data=[query_embedding],
    anns_field="embedding",
    param={"metric_type": "IP", "params": {"ef": 64}},
    limit=5,
    output_fields=["id", "name", "artists", "embedding_json"]
)

# Print results
for hit in search_results[0]:
    print(f"Score: {hit.score}")
    print(f"Name: {hit.entity.get('name')}")
    print(f"Artists: {hit.entity.get('artists')}")
    print("---")

Score: 456.99090576171875
Name: Shape of You
Artists: ['Ed Sheeran']
---
Score: 456.99090576171875
Name: Shape of You
Artists: ['Ed Sheeran']
---
Score: 413.04461669921875
Name: One
Artists: ['Ed Sheeran']
---
Score: 410.8623046875
Name: This
Artists: ['Ed Sheeran']
---
Score: 410.0792541503906
Name: Galway Girl
Artists: ['Ed Sheeran']
---
